<a href="https://colab.research.google.com/github/wujinja-cgu/Time-Series-Analysis/blob/main/Time%20series%20analysis%20by%20Pycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret
!pip install pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.1 MB/s e

In [ ]:
# read csv file
import pandas as pd
url = 'https://raw.githubusercontent.com/wujinja-cgu/Time-Series-Analysis/main/time%20series%20data.csv'
df = pd.read_csv(url)
df.head()

,DATE,No
0,10/09/2007,325
1,10/10/2007,417
2,10/11/2007,556
3,10/12/2007,467
4,10/13/2007,430


In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'])
data2 = df.copy()
data2 = data2[["DATE","No"]]
data2

,DATE,No
0,2007-10-09,325
1,2007-10-10,417
2,2007-10-11,556
3,2007-10-12,467
4,2007-10-13,430
...,...,...
2991,2015-12-17,503
2992,2015-12-18,444
2993,2015-12-19,438
2994,2015-12-20,471


In [ ]:
# create 30 days moving average
data2["MA"] = data2.rolling(window=7).mean()# plot the data and MA
import plotly.express as px
fig = px.line(data2, x="DATE", y=["No","MA"], template = 'plotly_dark')
fig.show()

<ipython-input-16-7f496464c399>:2: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['DATE'], dtype='object')
  data2["MA"] = data2.rolling(window=7).mean()# plot the data and MA


In [ ]:
import numpy as np
# extract month and year from dates
data2["Day"] = [i.day for i in data2["DATE"]]
data2["Month"] = [i.month for i in data2["DATE"]]
data2["Year"] = [i.year for i in data2["DATE"]]# create a sequence of numbers
data2["Series"] = np.arange(1,len(data2)+1)# drop unnecessary columns and re-arrange
data2.drop(["DATE", "MA"], axis=1, inplace=True)
data2 = data2[["Series", "Year", "Month", "Day", "No"]] # check the head of the dataset
data2.head()

,Series,Year,Month,Day,No
0,1,2007,10,9,325
1,2,2007,10,10,417
2,3,2007,10,11,556
3,4,2007,10,12,467
4,5,2007,10,13,430


In [ ]:
# split data into train-test set
train, test= np.split(data2, [int(.90 * len(data2))])
train.shape, test.shape

((2696, 5), (300, 5))

In [ ]:
!pip install markupsafe==2.0.1

  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.3
    Uninstalling MarkupSafe-2.1.3:
      Successfully uninstalled MarkupSafe-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
werkzeug 3.0.1 requires MarkupSafe>=2.1.1, but you have markupsafe 2.0.1 which is incompatible.


In [ ]:
# import the regression module
import jinja2
from pycaret.regression import *# initialize setup
s = setup(data = train, test_data = test, target = "No", fold_strategy = "timeseries", numeric_features = ["Month", "Series"], fold = 3, transform_target = True, session_id = 123, data_split_shuffle=False)

,Description,Value
0,Session id,123
1,Target,No
2,Target type,Regression
3,Original data shape,"(2996, 5)"
4,Transformed data shape,"(2996, 5)"
5,Transformed train set shape,"(2696, 5)"
6,Transformed test set shape,"(300, 5)"
7,Numeric features,2
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best=compare_models(sort="MAE")
best

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,41.3212,3538.9435,59.0854,-0.0703,0.1206,0.0883,0.0833
en,Elastic Net,41.3812,3533.5085,59.0470,-0.0691,0.1207,0.0886,0.0667
dummy,Dummy Regressor,41.3812,3533.5085,59.0470,-0.0691,0.1207,0.0886,0.0600
llar,Lasso Least Angle Regression,41.3812,3533.5085,59.0470,-0.0691,0.1207,0.0886,0.0733
lasso,Lasso Regression,41.3812,3533.5085,59.0470,-0.0691,0.1207,0.0886,0.0667
knn,K Neighbors Regressor,41.4846,3464.6463,58.5938,-0.0554,0.1201,0.0897,0.0533
ada,AdaBoost Regressor,43.2242,3653.7634,60.1634,-0.1131,0.1234,0.0923,0.0867
lightgbm,Light Gradient Boosting Machine,43.4838,3654.4771,60.3582,-0.1306,0.1238,0.0937,0.1733
et,Extra Trees Regressor,44.7574,3853.4051,61.9375,-0.1876,0.1274,0.0964,0.2633
rf,Random Forest Regressor,45.9773,3856.3701,62.0912,-0.2124,0.1284,0.1005,0.3800


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=123, ...)

In [ ]:
prediction_holdout = predict_model(best)
prediction_holdout

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,37.6505,2415.3528,49.1462,-0.1161,0.1035,0.0790


,Series,Year,Month,Day,No,prediction_label
2696,2697,2015,2,25,446,442.496338
2697,2698,2015,2,26,519,442.496338
2698,2699,2015,2,27,483,442.496338
2699,2700,2015,2,28,428,442.496338
2700,2701,2015,3,1,384,442.496338
...,...,...,...,...,...,...
2991,2992,2015,12,17,503,442.496338
2992,2993,2015,12,18,444,442.496338
2993,2994,2015,12,19,438,442.496338
2994,2995,2015,12,20,471,442.496338


In [ ]:
# generate predictions on the original dataset
predictions = predict_model(best, data=data2) # add a date column in the dataset
predictions

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,41.4928,3529.2838,59.4078,-0.0343,0.1209,0.0878


,Series,Year,Month,Day,No,prediction_label
0,1,2007,10,9,325,442.496338
1,2,2007,10,10,417,442.496338
2,3,2007,10,11,556,442.496338
3,4,2007,10,12,467,442.496338
4,5,2007,10,13,430,442.496338
...,...,...,...,...,...,...
2991,2992,2015,12,17,503,442.496338
2992,2993,2015,12,18,444,442.496338
2993,2994,2015,12,19,438,442.496338
2994,2995,2015,12,20,471,442.496338


In [ ]:
date_start = "2007-10-07"
date_end = "2015-12-19"
predictions["Date"] = pd.date_range(start = date_start, end = date_end) # line plot
fig = px.line(predictions, x="Date", y=["prediction_label","No"],template = "plotly_dark")# add a vertical rectange for test-set separation
fig.add_vrect(x0="2014–01–01", x1="2015–12–20", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()